In [1]:
from hhop import DFExtender, SchemaManager, write_table_through_view, spark, col, F
import shutil, os

from hhop import read_table # for testing

22/12/15 22:08:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/12/15 22:08:18 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Возможно, пригодятся 
1. .config("spark.hadoop.hive.exec.dynamic.partition", "true") \
2. .config("spark.hadoop.hive.exec.dynamic.partition.mode", "nonstrict") \

## Synth table

In [2]:
df_src = spark.read.csv('./synth_data/table1.csv', header=True, sep=';')

In [3]:
df_src.show()

+-----+----+----+------+--------+----------+----------+
|index| pk1| pk2|  var1|    var2|   dt_part|group_part|
+-----+----+----+------+--------+----------+----------+
|    1|key1|   1|value1|value2_1|2022-12-15|    group1|
|    2|key1|   1|value1|value2_1|2022-12-16|    group2|
|    3|key1|   2|value1|value2_1|2022-12-16|    group3|
|    4|key2|   2|  null|value2_1|2022-12-17|    group1|
|    5|key2|   3|value1|value2_1|2022-12-18|    group2|
|    6|key2|   4|value1|    null|2022-12-19|    group3|
|    7|key2|null|value1|value2_1|2022-12-19|    group4|
|    8|null|   4|value1|value2_1|2022-12-20|    group3|
|    9|null|null|value1|value2_1|2022-12-20|    group7|
+-----+----+----+------+--------+----------+----------+



In [4]:
df_src_write = df_src.write.mode('overwrite')

df_src_write.partitionBy('dt_part', 'group_part').saveAsTable('default.part_table_test1')
df_src_write.saveAsTable('default.nonpart_table_test1')

## Info about table

In [5]:
df = DFExtender('default','part_table_test1', pk=['pk1', 'pk2'])

table info default.part_table_test1
----------------------------------------
root
 |-- index: string (nullable = true)
 |-- pk1: string (nullable = true)
 |-- pk2: string (nullable = true)
 |-- var1: string (nullable = true)
 |-- var2: string (nullable = true)
 |-- dt_part: string (nullable = true)
 |-- group_part: string (nullable = true)

cols: ['index', 'pk1', 'pk2', 'var1', 'var2', 'dt_part', 'group_part']
partition columns: ['dt_part', 'group_part']
see attribute like this for full info: DataFrame.describe_table.show(100, False)

location: file:/Users/pyro/github/HiveHelper_on_PySpark/spark-warehouse/part_table_test1


In [6]:
df.describe_table.show(3, 30)

+--------+---------+-------+
|col_name|data_type|comment|
+--------+---------+-------+
|   index|   string|   null|
|     pk1|   string|   null|
|     pk2|   string|   null|
+--------+---------+-------+
only showing top 3 rows



### sanity checks: PK, NULL

## SchemaManager

In [7]:
schema_name = 'popular_schema'
spark.sql(f"drop database if exists {schema_name} cascade")
spark.sql(f'create database {schema_name}')

22/12/15 22:08:27 ERROR FileUtils: Failed to delete file:/Users/pyro/github/HiveHelper_on_PySpark/spark-warehouse/popular_schema.db/table3


DataFrame[]

In [8]:
df_src_write.partitionBy('dt_part', 'group_part').saveAsTable(f'{schema_name}.table1')
df_src_write.saveAsTable(f'{schema_name}.table2')
df_src_write.saveAsTable(f'{schema_name}.table3')

table1 has dir + data   
table2 has only root dir    
table3 doesn't have any dir and data

In [9]:
shutil.rmtree('./spark-warehouse/popular_schema.db/table2', ignore_errors=True)
os.makedirs('./spark-warehouse/popular_schema.db/table2')
shutil.rmtree('./spark-warehouse/popular_schema.db/table3', ignore_errors=True)

In [10]:
spark.sql(f"show tables in {schema_name}").show(10, False)

+--------------+---------+-----------+
|database      |tableName|isTemporary|
+--------------+---------+-----------+
|popular_schema|table1   |false      |
|popular_schema|table2   |false      |
|popular_schema|table3   |false      |
+--------------+---------+-----------+



In [11]:
list_of_tables = spark.sql(f"show tables in {schema_name}").select('tableName').rdd.flatMap(lambda x: x).collect()

In [12]:
read_table('popular_schema', 'table1').take(2)

[Row(index='1', pk1='key1', pk2='1', var1='value1', var2='value2_1', dt_part='2022-12-15', group_part='group1'),
 Row(index='2', pk1='key1', pk2='1', var1='value1', var2='value2_1', dt_part='2022-12-16', group_part='group2')]

## Comparing tables

## Modification